#Neural network coursework checker
This notebook is for checking ME4/MSc Machine Learning neural network coursework submission. You should run your model through this to check that you are getting the right answer prior to submission.
**If you get the wrong answers out, this is an indication that you need to change your model, not an indication that this script is incorrect.**

Also note that while this script may find many errors, it will not find all and that you are ultimately responsible for double-checking that what you submit is correct.

**Note that this script is not used for assessment (but note that similar routines are used to load the model). The data used is just to confirm that the model has been loaded properly.**

---
###Tidy up space (not normally needed)
The section below is just used for clearing any previously uploaded files. You should not normally need to run it, unless you have had crashes. If you run it you will need to re-upload the datasets. You can always open the file manager on the left hand side to look at the files too.

In [ ]:
#clear all files from the work area - only really need to do this if uploading
#files multiple times
!ls -lh
!rm *.csv
!rm *.h5
!rm *.txt

---
##Upload model and test with data
This section loads the files selected (note -- select both .h5 and .txt for scaling parameters if you have them, otherwise it will not do any scaling), and runs the training data you have uploaded through the model. You should check that this all works OK -- you don't get any errors -- and that the fraction correct matches what you saw when you trained it. Note that this only works with one model at a time so you will need to run it twice -- once each for dataset 1 and 2.

Also note that if you upload files with the same name multiple times then they will be renamed to e.g. xyz123-2 (1).h5 which will cause issues - the script tries to catch this by deleting any uploaded files at the end, but if it crashes you may need to delete all uploaded files using the section at the top.

In [2]:
#upload model files
import numpy as np
import os.path
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical

#import for bold font
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

#use this code in a non-Colab/Jupyter environment
# #user defined parameters (change these as necessary):
# #put your name here:
username = 'ae1220'
# #set which dataset to use:
dataset = 1

filebase = username.lower()+'-'+str(dataset)
scale_exists = True
h5file = 'ae1220-1.h5'
scalefile = 'ae1220-1.txt'
dataset = filebase[-1]
print('Dataset considered is: ',dataset)

#load in student model
model = load_model(h5file,compile=False)

if scale_exists:
  if os.path.exists(scalefile):
      print(scalefile+' exists - loading in scaling parameters')
      scaleArray = np.loadtxt(scalefile)
  else:
      print(scalefile+' not found - assuming no scaling')
      scaleArray = np.array([np.zeros([6,]), np.ones([6,])])
else:
    print('No scale file provided - assuming no scaling')
    scaleArray = np.array([np.zeros([6,]), np.ones([6,])])

#load in the data provided to the students
df = pd.read_csv('http://pogo.software/me4ml/dataset' + str(dataset) + '.csv')

Lt = np.array(df['Arm length (m)'][:])
Wt = np.array(df['Ball weight (kg)'][:])
Rt = np.array(df['Ball radius (mm)'][:])
Tt = np.array(df['Air temperature (deg C)'][:])
Et = np.array(df['Spring constant (N per m)'][:])
Dt = np.array(df['Device weight (kg)'][:])
Ot = np.array(df['Target hit'][:])
XtUnscaled = np.column_stack([Lt, Wt, Rt, Tt, Et, Dt])

# use values to scale validation data in XvUnscaled (whose shape is [number_of_validations,6])
Xt = (XtUnscaled-scaleArray[0,:])/scaleArray[1,:]

Yt = to_categorical(Ot)
#run the data through the model
Yt_predict = model.predict(Xt)

#output a summary of the model if you wish
#model.summary()


number_correct = 0
for i in range(len(Yt)):
    if np.round(Yt[i, 0]) == np.round(Yt_predict[i, 0]):
        number_correct += 1

fraction_correct = 1.0 * number_correct / len(Yt_predict)



printmd("**Fraction correct with training data is: "+str(fraction_correct)+"**")

if fraction_correct < 0.6:
    printmd('**Warning: very poor performance on training data; likely error**')

#print('Tidying up - removing h5 file and scale file if it exists')
os.remove(h5file)
if scale_exists:
  os.remove(scalefile)

Dataset considered is:  1
ae1220-1.txt exists - loading in scaling parameters
63/63 [==============================] - 0s 2ms/step


**Fraction correct with training data is: 0.9225**